# Data Analysis using Pandas


### What is Pandas?
- Pandas is an extremely popular library for data analysis - you can generally think of it as an Excel replacement.
- Allows tabular data to be efficiently read, manipulated, and analysed
- It is able to read and write data from various filetypes
- Includes powerful methods to slice, reshape and mutate datasets
- Provides relatively simple ways for dealing with dates/time and missing data

### Why use Pandas?
- As dataset are becoming bigger and more complex, tools such as pandas are becoming increasingly essential.
- Anlysis using Python and Pandas is generally more transparent and easier to follow than that done in speadsheets.
- It also encourage some seperation of data and analysis.
- Easy to automate and apply to multiple datasets.

In [ ]:
import pandas as pd

### Creating a Dataframe from spreadsheet data
>* Pandas allows reading importing data from an Excel/csv file into a pandas DataFrame.
>* We can read a CSV file with pandas by using `pd.read_csv`. Excel files can be read using `pd.read_excel`.

In [ ]:
filepath = r"data\Conowingo_data.xlsx" # r is to treat all characters between the quotes as a raw string.
                                  # Otherwise, special couples, such as "\n", may be treated as a new line

# Read specific extent (columns and rows) from a specific worksheet
demand_df = pd.read_excel(filepath, 
                   sheet_name='Demands', # Specify the worksheet
                   usecols="A:E",        # Specify columns (e.g., columns A to E)
                   skiprows=0,           # (Optional) Skip the first n row if necessary (skiprows=0 does not skip a row)
                   nrows=12,             # (Optional) Read the first 12 rows after the first row (first row will be column titles)
                   index_col=0)          # Dataframe index column (0 indicates the first column)
demand_df

In [ ]:
# Display only the first n rows of the dataframe
demand_df.head(3)

In [ ]:
# Display only the last n rows of the dataframe
demand_df.tail(2)

### Quyerying a DataFramme
> * `DataFrame.index` will return information about the columns names
* `DataFrame.columns` will also return row names of in the case, a list of the dates

In [ ]:
print(demand_df.columns)
print("----------")
print(demand_df.index)

### Selecting values

Referencing one column by using `DataFrame[name_of_column]`

In [ ]:
demand_df["Peach Bottom Nuclear Power Plant (cfs)"]

Subsetting a DataFrame by using `DataFrame.loc`. This selection finds a value within the DataFrame below `df.loc[index, column]` where index and column are their **labels**.

In [ ]:
demand_df.loc["June", "Minimum environmental flows (cfs)"]

In [ ]:
# To display only number, use print command
print(demand_df.loc["June", "Minimum environmental flows (cfs)"])

Use a : when you want to select all the values for either rows or columns

In [ ]:
demand_df.loc[:, "Minimum environmental flows (cfs)"]

In [ ]:
demand_df.loc["June", :]

Subsetting a DataFrame by using `DataFrame.iloc`. This selection finds a values as above but now index and column are instead their **positions**. Please note: In python, position indexing starts from 0.

In [ ]:
demand_df.iloc[6, 2]

Subsetting a DataFrame with a range of criteria

In [ ]:
demand_df.loc["March":"May", "Chester city (cfs)":"Peach Bottom Nuclear Power Plant (cfs)"]

In [ ]:
demand_df.loc[["November", "March", "August", "May"], ["Peach Bottom Nuclear Power Plant (cfs)","Chester city (cfs)"]]

In [ ]:
demand_df.iloc[2:5, 1:3] # Display the slice of the dataframe from 3rd to 5th row and 2nd to 3rd column

### Operations of Dataframes and Columns

In [ ]:
# Create a deepcopy of the DataFrame. A deepcopied dataframe is totally independent from the original dataframe.
demand_m3 = demand_df.copy()

# Now, any changes to 'demand_m3' won't affect 'demand_df'

In [ ]:
# Convert all values in the dataframe to metric system units: conversion from cfs to m3/s
demand_m3 = demand_m3 * 0.3048**3

# We need to also change "cfs" with "m3/s" in column headers
demand_m3.columns = pd.Series(demand_m3.columns).replace('cfs', 'm3/s', regex=True)

# What does that look like, rounded to the first decimal place?
demand_m3.round(1)

In [ ]:
# Calculate the sum of values across each row in the dataframe (for total monthly demand)
# Calculated sum will be added as a new column to the copied dataframe
demand_m3["Total Demand (m3/s)"]  = demand_m3.sum(axis=1)
demand_m3.round(1).head()

In [ ]:
demand_m3["Total Potable Water Demand (m3/s)"]  = demand_m3["Chester city (m3/s)"] + demand_m3["Baltimore city (m3/s)"]
demand_m3.round(1).head()

In [ ]:
# Calculate the average volumetric demand across each column, throughout the year
annual_means = demand_m3.mean(axis=0)

# Print the result
print(annual_means.round(1))

In [ ]:
annual_means["Total Demand (m3/s)"].round(1)

In [ ]:
# Calculate the percent contribution for each demand to the annual total demand
percent_contribution = (annual_means.iloc[:4] / annual_means["Total Demand (m3/s)"]) * 100

# Print the result
print(percent_contribution)

# Remove a column

In [ ]:
# Need to set axis=1 overwise drop will try to remove a row as the default is axis=0
demand_m3 = demand_m3.drop("Total Potable Water Demand (m3/s)", axis=1)
demand_m3.round(1).head()

In [ ]:
# Remove another column
demand_m3 = demand_m3.drop("Total Demand (m3/s)", axis=1)
demand_m3.head()

### Working with timeseries data
- We often work with data with a temporal axis, pandas makes it easy to deal with this
- We normally set the time-axis as the index
- Control for reading in datasets, skip rows and columns 

### We can set date to be the index column and recognise this as a DateTimeIndex
 - Use `index_col` to set the column you want
 - The output of `read_csv` is a DataFrame

In [ ]:
# Read specific extent (columns and rows) from a specific worksheet
flow_df = pd.read_excel(filepath, 
                   sheet_name='Flow data', # Specify the worksheet
                   index_col=0)            # Dataframe index column (0 indicates the 1st column)



flow_df.head()

In [ ]:
import matplotlib.pyplot as plt
flow_df.loc[:,["Susquehanna River Inflow (cfs)","Lateral Inflow (cfs)"]].plot()
plt.show()

Clearly river inflows to the reservoirs are the bulk of the total inflows. Let's have a new, total inflow column using SI units.

In [ ]:
flow_df['Total inflows (m3/s)'] = flow_df.sum(axis=1) * 0.3048**3

flow_df.loc[:, ['Total inflows (m3/s)']].plot()

In [ ]:
# Plot only a subset in the dataframe
flow_df.loc["1991-3-1":"1991-5-31",['Total inflows (m3/s)']].plot()
plt.show()

In [ ]:
# You can also customise the visuals if you want!
import datetime

def plot_flux(balance, flux_name, **kwargs):
    """
    Plots daily timeseries of a water balance flow component over time. Arguments:
        balance: a Pandas DataFrame containing the time series of the water flux to plot
        flux_name: a string with the name of the flow component to plot
        optional argument `first_date`: a datetime date to specify when to start plotting
        optional argument `last_date`: a datetime date to specify when to stop plotting
    Returns the matplotlib figure created, for plotting / saving, etc.
    """

    # Optional arguments
    first_date = kwargs.pop("first_date", balance.index[0])
    last_date = kwargs.pop('last_date', balance.index[-1])

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(balance.index, balance[flux_name + ' (m3/s)'], c='b', linewidth=2)
    ax.set_xlabel('Date', size=14)
    ax.set_ylabel(flux_name + ' (m3/s)', size=14)
    ax.set_xlim(first_date, last_date)
    ax.set_ylim(0, balance.loc[first_date : last_date, flux_name + ' (m3/s)'].max() * 1.1)

    return fig

fig = plot_flux(flow_df, 'Total inflows', first_date=datetime.date(1964, 1, 1), last_date=datetime.date(1964,12,31))

### Filter for values above a threshold

In [ ]:
mask = flow_df['Total inflows (m3/s)'] > 500
mask

In [ ]:
flow_df[mask]

# Resample a DataFrame using the datetime index

In [ ]:
# Resample the dataframe into annual average flows
flow_df['Total inflows (m3/s)'].resample("YE").mean().plot()

In [ ]:
# Resample the dataframe into monthly mean flow
flow_df['Total inflows (m3/s)'].resample("1ME").mean().plot()

In [ ]:
import numpy as np

# But what does that look like on average
def compute_monthly_average(flows, variable_name):
    """
    Computes monthly average inflows from a `flows` pandas DataFrame containing a 'Total inflows (m3/s)' column.
    Output:
    averages: a Numpy vector of size 12 for the 12 average monthly values
    """

    # Initialise output
    averages = np.zeros(12)

    # Main loop to compute all 12 monthly averages
    for month in np.arange(1, 13, 1):
        monthly_mask = flows.index.month == month  # Select only values for the right month
        averages[month - 1] = flows.loc[monthly_mask, variable_name].mean()  # Apply average operator

    return averages


# Plotting the figure
def monthly_averages(flows, variable_name):
    """
    Plot monthly average inflows from `flows` pandas DataFrame.
    """

    # Get monthly average inflows
    monthly_average = compute_monthly_average(flows, variable_name)

    # Plot figure
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.arange(1, 13, 1), monthly_average, c='b')
    plt.xticks(ticks=np.arange(1, 13, 1), labels=['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    ax.set_xlabel('Month', size=14)
    ax.set_ylabel('Average inflows (m3/s)', size=14)
    ax.set_xlim(1, 12)

    return fig

fig = monthly_averages(flow_df, 'Total inflows (m3/s)')

Did you notice we just introduced the numpy library?

# What is numpy?
- Numpy is a python library supporting the use of ***multi-dimensional arrays*** and ***simple mathematical functions***
- Numpy is used to efficiently carry out various mathematical operations on large data structures
- Open source with some minor restrictions (?)
- The pandas library is built on top of numpy

### What is an array?
An array is a central data structure of the NumPy library. An array is a grid of values and it contains information about the raw data, how to locate an element, and how to interpret an element. It has a grid of elements that can be indexed in various ways. The elements are all of the same type, referred to as the array dtype.

An array can be indexed by a tuple of nonnegative integers, by booleans, by another array, or by integers. The rank of the array is the number of dimensions. The shape of the array is a tuple of integers giving the size of the array along each dimension.

One way we can initialize NumPy arrays is from Python lists, using nested lists for two- or higher-dimensional data.

For example:

In [ ]:
a = np.array([1, 2, 3, 4, 5, 6])
print(a)

In [ ]:
a2 = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
print(a2)

### Creating a DataFrame using a numpy array

In [ ]:
columns = ["Column 1", "Column 2", "Column 3", "Column 4"]
rows = ["R1", "R2", "R3"]
df2 = pd.DataFrame(a2, columns=columns, index=rows)
df2

### Acessing the underlying numpy array of a Pandas DataFrame

In [ ]:
demand_df.values

In [ ]:
type(demand_df.values)

 - Numpy docs have more infomation on usage for beginners 
 - https://numpy.org/devdocs/user/absolute_beginners.html